# init

## import Python modules

In [1]:
import pandas, IPython.core.display

## open file

In [2]:
src = pandas.ExcelFile('2017 CAM data from iPads.xlsx')
src.sheet_names

['2017 CAM data Erl',
 'schema (WIP reverse engineer)',
 '2017 CAM iPad data Tyler',
 'Combined iPad 2017 CAM data']

## select sheets

In [3]:
sheets_to_process = {
    sheet_name.split(' ')[-1]: src.parse(sheet_name)
    for sheet_name in src.sheet_names
    if any(
        (
            person_name in sheet_name
            for person_name
            in ['Erl', 'Tyler']
        )
    )
}
sheets_to_process.keys()

dict_keys(['Erl', 'Tyler'])

# cleanup

## normalize column names

In [4]:
sheets_needing_column_rename = (
    (sheet_name, sheet)
    for sheet_name, sheet
    in sheets_to_process.items()
    if any(
        (str(column).find(' ')
         for column in sheet.columns)
    )
)
sheets_to_process = {
    sheet_name: sheet.rename(
        mapper=lambda x: str(x).split(' ')[0],
        axis='columns')
    for sheet_name, sheet in sheets_needing_column_rename
}

## concatenate sheets

In [5]:
df = pandas.concat(sheets_to_process,
                   names=['worksheet_name', 'index'])

## reindex columns

In [6]:
names = ['field', 'scope', 'variable']
columns = (
    ('fields__oSets__date',                              'set', 'date'),
    ('fields__oSets__oPoints__id',                       'point', 'id'),
    ('fields__oSets__oPoints__observations__id',         'observation', 'id'),
    ('fields__oSets__oPoints__observations__enum',       'point', 'natural_enemy_subtotal'),
    ('fields__oSets__oPoints__observations__|',          'natural_enemy_count', 'name'),
    ('fields__oSets__oPoints__observations__|__number',  'natural_enemy_count', 'count'),
    ('fields__oSets__oPoints__observations__complete',   'observation', 'complete'),
    ('fields__oSets__oPoints__observations__disabled',   'observation', 'disabled'),
    ('fields__oSets__oPoints__observations__a1__number', 'aphid_count', 'a1'),
    ('fields__oSets__oPoints__observations__a2__number', 'aphid_count', 'a2'),
    ('fields__oSets__oPoints__observations__a3__number', 'aphid_count', 'a3'),
    ('fields__oSets__oPoints__observations__anum',       'observation', 'aphid_subtotal'),
    ('fields__oSets__obsName',                           'set', 'obsName'),
    ('fields__oSets__desc',                              'set', 'desc'),
    ('fields__oSets__growthStage',                       'set', 'growthStage'),
    ('fields__oSets__results',                           'set_results', 'results'),
    ('fields__oSets__totalA1',                           'set_results', 'totalA1'),
    ('fields__oSets__totalA2',                           'set_results', 'totalA2'),
    ('fields__oSets__totalA3',                           'set_results', 'totalA3'),
    ('fields__oSets__totalA4',                           'set_results', 'totalA4'),
    ('fields__oSets__totalSets',                         'set_results', 'totalSets'),
    ('fields__oSets__completeSets',                      'set_results', 'completeSets'),
    ('fields__client__company',                          'field', 'client_company'),
    ('fields__client__displayText',                      'field', 'client_displayText'),
    ('fields__client__fname',                            'field', 'client_fname'),
    ('fields__client__lname',                            'field', 'client_lname'),
    ('fields__client__name',                             'field', 'client_name'),
    ('fields__crop',                                     'field', 'crop'),
    ('fields__date',                                     'field', 'date'),
    ('fields__desc',                                     'field', 'desc'),
    ('fields__image',                                    'field', 'image'),
    ('fields__name',                                     'field', 'name'),
)
columns = pandas.MultiIndex.from_tuples(columns, names=names)
df = df.reindex(columns=columns, level='field')
df.columns = df.columns.droplevel(level='field')

## convert_datetime

In [7]:
for column in [('set','date'), ('field','date')]:
    df[column] = pandas.to_datetime(df[column], infer_datetime_format=True)

## forward fill

In [8]:
fill_columns = [
    ('point', 'id'),
    ('observation', 'id'),
] + list(df[['field', 'set']].columns.values)
df.loc[:, fill_columns] = df.loc[:, fill_columns].ffill()

In [9]:
df = df.set_index(([('set', 'date'), ('point', 'id')])).sort_index().reset_index()
df

scope                        set point observation                  point  \
variable                    date    id          id natural_enemy_subtotal   
0        2017-07-14 12:31:24.194   0.0         0.0                    NaN   
1        2017-07-14 12:31:24.194   0.0         1.0                    NaN   
2        2017-07-14 12:31:24.194   0.0         2.0                    NaN   
3        2017-07-14 12:31:24.194   0.0         3.0                    NaN   
4        2017-07-14 12:31:24.194   0.0         4.0                    NaN   
5        2017-07-14 12:31:24.194   0.0         5.0                    2.0   
6        2017-07-14 12:31:24.194   0.0         5.0                    NaN   
7        2017-07-14 12:31:24.194   0.0         5.0                    NaN   
8        2017-07-14 12:31:24.194   0.0         5.0                    NaN   
9        2017-07-14 12:31:24.194   0.0         5.0                    NaN   
10       2017-07-14 12:31:24.194   0.0         5.0                    NaN   
11       2017-07-14 12:31:24.194   0.0         5.0                    NaN   
12       2017-07-14 12:31:24.194   0.0         5.0                    NaN   
13       2017-07-14 12:31:24.194   0.0         5.0                    NaN   
14       2017-07-14 12:31:24.194   1.0         0.0                    NaN   
15       2017-07-14 12:31:24.194   1.0         1.0                    NaN   
16       2017-07-14 12:31:24.194   1.0         2.0                    NaN   
17       2017-07-14 12:31:24.194   1.0         3.0                    NaN   
18       2017-07-14 12:31:24.194   1.0         4.0                    NaN   
19       2017-07-14 12:31:24.194   1.0         5.0                    0.0   
20       2017-07-14 12:31:24.194   1.0         5.0                    NaN   
21       2017-07-14 12:31:24.194   1.0         5.0                    NaN   
22       2017-07-14 12:31:24.194   1.0         5.0                    NaN   
23       2017-07-14 12:31:24.194   1.0         5.0                    NaN   
24       2017-07-14 12:31:24.194   1.0         5.0                    NaN   
25       2017-07-14 12:31:24.194   1.0         5.0                    NaN   
26       2017-07-14 12:31:24.194   1.0         5.0                    NaN   
27       2017-07-14 12:31:24.194   1.0         5.0                    NaN   
28       2017-07-14 12:31:24.194   2.0         0.0                    NaN   
29       2017-07-14 12:31:24.194   2.0         1.0                    NaN   
...                          ...   ...         ...                    ...   
4660     2017-08-25 13:05:14.062   7.0         5.0                    NaN   
4661     2017-08-25 13:05:14.062   7.0         5.0                    NaN   
4662     2017-08-25 13:05:14.062   8.0         0.0                    NaN   
4663     2017-08-25 13:05:14.062   8.0         1.0                    NaN   
4664     2017-08-25 13:05:14.062   8.0         2.0                    NaN   
4665     2017-08-25 13:05:14.062   8.0         3.0                    NaN   
4666     2017-08-25 13:05:14.062   8.0         4.0                    NaN   
4667     2017-08-25 13:05:14.062   8.0         5.0                    0.0   
4668     2017-08-25 13:05:14.062   8.0         5.0                    NaN   
4669     2017-08-25 13:05:14.062   8.0         5.0                    NaN   
4670     2017-08-25 13:05:14.062   8.0         5.0                    NaN   
4671     2017-08-25 13:05:14.062   8.0         5.0                    NaN   
4672     2017-08-25 13:05:14.062   8.0         5.0                    NaN   
4673     2017-08-25 13:05:14.062   8.0         5.0                    NaN   
4674     2017-08-25 13:05:14.062   8.0         5.0                    NaN   
4675     2017-08-25 13:05:14.062   8.0         5.0                    NaN   
4676     2017-08-25 13:05:14.062   9.0         0.0                    NaN   
4677     2017-08-25 13:05:14.062   9.0         1.0                    NaN   
4678     2017-08-25 13:05:14.062   9.0         2.0                    NaN

## split frame into parts

### non-aggregate

#### index

In [10]:
index_columns = list(df[[('set', 'date'), ('point', 'id'), ('observation', 'id')]].columns)
index_columns

[('set', 'date'), ('point', 'id'), ('observation', 'id')]

In [11]:
index_df = df.loc[
    df[('observation', 'aphid_subtotal')].notna() | df[('point', 'natural_enemy_subtotal')].notna(),
    index_columns
]
index_df = index_df.set_index(index_columns)
index_df

Empty DataFrame
Columns: []
Index: [(2017-07-14 12:31:24.194000, 0.0, 0.0), (2017-07-14 12:31:24.194000, 0.0, 1.0), (2017-07-14 12:31:24.194000, 0.0, 2.0), (2017-07-14 12:31:24.194000, 0.0, 3.0), (2017-07-14 12:31:24.194000, 0.0, 4.0), (2017-07-14 12:31:24.194000, 0.0, 5.0), (2017-07-14 12:31:24.194000, 1.0, 0.0), (2017-07-14 12:31:24.194000, 1.0, 1.0), (2017-07-14 12:31:24.194000, 1.0, 2.0), (2017-07-14 12:31:24.194000, 1.0, 3.0), (2017-07-14 12:31:24.194000, 1.0, 4.0), (2017-07-14 12:31:24.194000, 1.0, 5.0), (2017-07-14 12:31:24.194000, 2.0, 0.0), (2017-07-14 12:31:24.194000, 2.0, 1.0), (2017-07-14 12:31:24.194000, 2.0, 2.0), (2017-07-14 12:31:24.194000, 2.0, 3.0), (2017-07-14 12:31:24.194000, 2.0, 4.0), (2017-07-14 12:31:24.194000, 2.0, 5.0), (2017-07-14 12:31:24.194000, 3.0, 0.0), (2017-07-14 12:31:24.194000, 3.0, 1.0), (2017-07-14 12:31:24.194000, 3.0, 2.0), (2017-07-14 12:31:24.194000, 3.0, 3.0), (2017-07-14 12:31:24.194000, 3.0, 4.0), (2017-07-14 12:31:24.194000, 3.0, 5.0), (2017-07-14 12:31:24.194000, 4.0, 0.0), (2017-07-14 12:31:24.194000, 4.0, 1.0), (2017-07-14 12:31:24.194000, 4.0, 2.0), (2017-07-14 12:31:24.194000, 4.0, 3.0), (2017-07-14 12:31:24.194000, 4.0, 4.0), (2017-07-14 12:31:24.194000, 4.0, 5.0), (2017-07-18 10:31:22.263000, 0.0, 0.0), (2017-07-18 10:31:22.263000, 0.0, 1.0), (2017-07-18 10:31:22.263000, 0.0, 2.0), (2017-07-18 10:31:22.263000, 0.0, 3.0), (2017-07-18 10:31:22.263000, 0.0, 4.0), (2017-07-18 10:31:22.263000, 0.0, 5.0), (2017-07-18 10:31:22.263000, 1.0, 0.0), (2017-07-18 10:31:22.263000, 1.0, 1.0), (2017-07-18 10:31:22.263000, 1.0, 2.0), (2017-07-18 10:31:22.263000, 1.0, 3.0), (2017-07-18 10:31:22.263000, 1.0, 4.0), (2017-07-18 10:31:22.263000, 1.0, 5.0), (2017-07-18 10:31:22.263000, 2.0, 0.0), (2017-07-18 10:31:22.263000, 2.0, 1.0), (2017-07-18 10:31:22.263000, 2.0, 2.0), (2017-07-18 10:31:22.263000, 2.0, 3.0), (2017-07-18 10:31:22.263000, 2.0, 4.0), (2017-07-18 10:31:22.263000, 2.0, 5.0), (2017-07-18 10:31:22.263000, 3.0, 0.0), (2017-07-18 10:31:22.263000, 3.0, 1.0), (2017-07-18 10:31:22.263000, 3.0, 2.0), (2017-07-18 10:31:22.263000, 3.0, 3.0), (2017-07-18 10:31:22.263000, 3.0, 4.0), (2017-07-18 10:31:22.263000, 3.0, 5.0), (2017-07-18 10:31:22.263000, 4.0, 0.0), (2017-07-18 10:31:22.263000, 4.0, 1.0), (2017-07-18 10:31:22.263000, 4.0, 2.0), (2017-07-18 10:31:22.263000, 4.0, 3.0), (2017-07-18 10:31:22.263000, 4.0, 4.0), (2017-07-18 10:31:22.263000, 4.0, 5.0), (2017-07-18 10:51:30.195000, 0.0, 0.0), (2017-07-18 10:51:30.195000, 0.0, 1.0), (2017-07-18 10:51:30.195000, 0.0, 2.0), (2017-07-18 10:51:30.195000, 0.0, 3.0), (2017-07-18 10:51:30.195000, 0.0, 4.0), (2017-07-18 10:51:30.195000, 0.0, 5.0), (2017-07-18 10:51:30.195000, 1.0, 0.0), (2017-07-18 10:51:30.195000, 1.0, 1.0), (2017-07-18 10:51:30.195000, 1.0, 2.0), (2017-07-18 10:51:30.195000, 1.0, 3.0), (2017-07-18 10:51:30.195000, 1.0, 4.0), (2017-07-18 10:51:30.195000, 1.0, 5.0), (2017-07-18 10:51:30.195000, 2.0, 0.0), (2017-07-18 10:51:30.195000, 2.0, 1.0), (2017-07-18 10:51:30.195000, 2.0, 2.0), (2017-07-18 10:51:30.195000, 2.0, 3.0), (2017-07-18 10:51:30.195000, 2.0, 4.0), (2017-07-18 10:51:30.195000, 2.0, 5.0), (2017-07-18 10:51:30.195000, 3.0, 0.0), (2017-07-18 10:51:30.195000, 3.0, 1.0), (2017-07-18 10:51:30.195000, 3.0, 2.0), (2017-07-18 10:51:30.195000, 3.0, 3.0), (2017-07-18 10:51:30.195000, 3.0, 4.0), (2017-07-18 10:51:30.195000, 3.0, 5.0), (2017-07-18 10:51:30.195000, 4.0, 0.0), (2017-07-18 10:51:30.195000, 4.0, 1.0), (2017-07-18 10:51:30.195000, 4.0, 2.0), (2017-07-18 10:51:30.195000, 4.0, 3.0), (2017-07-18 10:51:30.195000, 4.0, 4.0), (2017-07-18 10:51:30.195000, 4.0, 5.0), (2017-07-18 11:39:46.221000, 0.0, 0.0), (2017-07-18 11:39:46.221000, 0.0, 1.0), (2017-07-18 11:39:46.221000, 0.0, 2.0), (2017-07-18 11:39:46.221000, 0.0, 3.0), (2017-07-18 11:39:46.221000, 0.0, 4.0), (2017-07-18 11:39:46.221000, 0.0, 5.0), (2017-07-18 11:39:46.221000, 1.0, 0.0), (2017-07-18 11:39:46.221000, 1.0, 1.0), (2017-07-18 11:39:46.221000, 1.0, 2.0), (201

#### categorical

In [12]:
base_df_column_names = list(df[[
#     ('set', 'date'),
#     ('observation', 'complete'),
#     ('observation', 'disabled'),
    ('point', 'id'),
    ('observation', 'id'),
]].columns) + list(df[[
    'field',
    'set',
]].columns)
base_df = df.loc[
    df[('observation', 'aphid_subtotal')].notna() | df[('point', 'natural_enemy_subtotal')].notna(),
    base_df_column_names
]
base_df = base_df.set_index(index_columns)
base_df

scope                                                          field  \
variable                                              client_company   
(set, date)             (point, id) (observation, id)                  
2017-07-14 12:31:24.194 0.0         0.0                    AAFC SRDC   
                                    1.0                    AAFC SRDC   
                                    2.0                    AAFC SRDC   
                                    3.0                    AAFC SRDC   
                                    4.0                    AAFC SRDC   
                                    5.0                    AAFC SRDC   
                        1.0         0.0                    AAFC SRDC   
                                    1.0                    AAFC SRDC   
                                    2.0                    AAFC SRDC   
                                    3.0                    AAFC SRDC   
                                    4.0                    AAFC SRDC   
                                    5.0                    AAFC SRDC   
                        2.0         0.0                    AAFC SRDC   
                                    1.0                    AAFC SRDC   
                                    2.0                    AAFC SRDC   
                                    3.0                    AAFC SRDC   
                                    4.0                    AAFC SRDC   
                                    5.0                    AAFC SRDC   
                        3.0         0.0                    AAFC SRDC   
                                    1.0                    AAFC SRDC   
                                    2.0                    AAFC SRDC   
                                    3.0                    AAFC SRDC   
                                    4.0                    AAFC SRDC   
                                    5.0                    AAFC SRDC   
                        4.0         0.0                    AAFC SRDC   
                                    1.0                    AAFC SRDC   
                                    2.0                    AAFC SRDC   
                                    3.0                    AAFC SRDC   
                                    4.0                    AAFC SRDC   
                                    5.0                    AAFC SRDC   
...                                                              ...   
2017-08-25 13:05:14.062 5.0         0.0                    AAFC SRDC   
                                    1.0                    AAFC SRDC   
                                    2.0                    AAFC SRDC   
                                    3.0                    AAFC SRDC   
                                    4.0                    AAFC SRDC   
                                    5.0                    AAFC SRDC   
                        6.0         0.0                    AAFC SRDC   
                                    1.0                    AAFC SRDC   
                                    2.0                    AAFC SRDC   
                                    3.0                    AAFC SRDC   
                                    4.0                    AAFC SRDC   
                                    5.0                    AAFC SRDC   
                        7.0         0.0                    AAFC SRDC   
                                    1.0                    AAFC SRDC   
                                    2.0                    AAFC SRDC   
                                    3.0                    AAFC SRDC   
                                    4.0                    AAFC SRDC   
                                    5.0                    AAFC SRDC   
                        8.0         0.0                    AAFC SRDC   
                                    1.0                    AAFC SRDC   
                                    2.0                    AAFC SRDC   
                                    3.0 

### aphid

In [13]:
aphid_columns = index_columns + list(df[[
    ('observation', 'complete'),
    ('observation', 'disabled'),
]]) + list(df[[
    'aphid_count'
]].columns)
aphid_df = df.loc[df[('observation', 'id')] < 5, aphid_columns].set_index(index_columns).fillna(0)
aphid_df

scope                                                 observation           \
variable                                                 complete disabled   
(set, date)             (point, id) (observation, id)                        
2017-07-14 12:31:24.194 0.0         0.0                       0.0      0.0   
                                    1.0                       0.0      0.0   
                                    2.0                       0.0      0.0   
                                    3.0                       0.0      0.0   
                                    4.0                       0.0      0.0   
                        1.0         0.0                       0.0      0.0   
                                    1.0                       0.0      0.0   
                                    2.0                       0.0      0.0   
                                    3.0                       0.0      0.0   
                                    4.0                       0.0      0.0   
                        2.0         0.0                       0.0      0.0   
                                    1.0                       0.0      0.0   
                                    2.0                       0.0      0.0   
                                    3.0                       0.0      0.0   
                                    4.0                       0.0      0.0   
                        3.0         0.0                       0.0      0.0   
                                    1.0                       0.0      0.0   
                                    2.0                       0.0      0.0   
                                    3.0                       0.0      0.0   
                                    4.0                       0.0      0.0   
                        4.0         0.0                       0.0      0.0   
                                    1.0                       0.0      0.0   
                                    2.0                       0.0      0.0   
                                    3.0                       0.0      0.0   
                                    4.0                       0.0      0.0   
2017-07-18 10:31:22.263 0.0         0.0                       1.0      0.0   
                                    1.0                       1.0      0.0   
                                    2.0                       1.0      0.0   
                                    3.0                       1.0      0.0   
                                    4.0                       1.0      0.0   
...                                                           ...      ...   
2017-08-25 13:05:14.062 4.0         0.0                       1.0      0.0   
                                    1.0                       1.0      0.0   
                                    2.0                       1.0      0.0   
                                    3.0                       1.0      0.0   
                                    4.0                       1.0      0.0   
                        5.0         0.0                       1.0      0.0   
                                    1.0                       1.0      0.0   
                                    2.0                       1.0      0.0   
                                    3.0                       1.0      0.0   
                                    4.0                       1.0      0.0   
                        6.0         0.0                       1.0      0.0   
                                    1.0                       1.0      0.0   
                                    2.0                       1.0      0.0   
                                    3.0                       1.0      0.0   
                                    4.0                       1.0      0.0   
                        7.0         0.0                       1.0      0.0   
                                    1.0                       1.0      0.0   
                      

### natural enemies

In [14]:
ne_columns = list(df[[('set', 'date'), ('point', 'id'), ('observation', 'complete'), ('observation', 'disabled')]].columns) + list(df[['natural_enemy_count']].columns)
ne = df.loc[:, ne_columns]
ne = ne.dropna(subset=df[[('natural_enemy_count', 'name')]].columns)
ne = ne.reset_index()
ne = ne.set_index([('set', 'date'), ('point', 'id'), ('natural_enemy_count', 'name')]).fillna(0)
ne

scope                                                           index  \
variable                                                                
(set, date)             (point, id) (natural_enemy_count, name)         
2017-07-14 12:31:24.194 0.0         e1                              5   
                                    e2                              6   
                                    e3                              7   
                                    e4                              8   
                                    e5                              9   
                                    e6                             10   
                                    e7                             11   
                                    e8                             12   
                                    e9                             13   
                        1.0         e1                             19   
                                    e2                             20   
                                    e3                             21   
                                    e4                             22   
                                    e5                             23   
                                    e6                             24   
                                    e7                             25   
                                    e8                             26   
                                    e9                             27   
                        2.0         e1                             33   
                                    e2                             34   
                                    e3                             35   
                                    e4                             36   
                                    e5                             37   
                                    e6                             38   
                                    e7                             39   
                                    e8                             40   
                                    e9                             41   
                        3.0         e1                             47   
                                    e2                             48   
                                    e3                             49   
...                                                               ...   
2017-08-25 13:05:14.062 6.0         e7                           4645   
                                    e8                           4646   
                                    e9                           4647   
                        7.0         e1                           4653   
                                    e2                           4654   
                                    e3                           4655   
                                    e4                           4656   
                                    e5                           4657   
                                    e6                           4658   
                                    e7                           4659   
                                    e8                           4660   
                                    e9                           4661   
                        8.0         e1                           4667   
                                    e2                           4668   
                                    e3                           4669   
                                    e4                           4670   
                                    e5                           4671   
                                    e6                           4672   
                                    e7                           4673   
                                    e8                           4674   
                                    e9                    

#### natural enemy transpose

In [15]:
ne2 = ne.unstack().reset_index()
ne2['index_base'] = ne2.pop(('index','','e1'))
ne2[('observation', 'complete', '')] = ne2.pop(('observation', 'complete','e1'))
ne2[('observation', 'disabled', '')] = ne2.pop(('observation', 'disabled','e1'))

In [18]:
ne2.head()

scope                                               set point index          \
variable                                           date    id                 
('natural_enemy_count', 'name')                                  e2  e3  e4   
0                               2017-07-14 12:31:24.194   0.0     6   7   8   
1                               2017-07-14 12:31:24.194   1.0    20  21  22   
2                               2017-07-14 12:31:24.194   2.0    34  35  36   
3                               2017-07-14 12:31:24.194   3.0    48  49  50   
4                               2017-07-14 12:31:24.194   4.0    62  63  64   

scope                                                 ...     \
variable                                              ...      
('natural_enemy_count', 'name')  e5  e6  e7  e8  e9   ...      
0                                 9  10  11  12  13   ...      
1                                23  24  25  26  27   ...      
2                                37  38  39  40  41   ...      
3                                51  52  53  54  55   ...      
4                                65  66  67  68  69   ...      

scope                           natural_enemy_count                           \
variable                                      count                            
('natural_enemy_count', 'name')                  e3   e4   e5   e6   e7   e8   
0                                               0.0  0.0  0.0  0.0  0.0  0.0   
1                                               0.0  0.0  0.0  0.0  0.0  0.0   
2                                               0.0  0.0  0.0  0.0  0.0  0.0   
3                                               0.0  0.0  0.0  0.0  0.0  0.0   
4                                               0.0  0.0  0.0  0.0  0.0  0.0   

scope                                index_base observation           
variable                                           complete disabled  
('natural_enemy_count', 'name')   e9                                  
0                                0.0          5         1.0      0.0  
1                                0.0         19         1.0      0.0  
2                                0.0         33         1.0      0.0  
3                                0.0         47         1.0      0.0  
4                                0.0         61         1.0      0.0  

[5 rows x 38 columns]

In [23]:
ne3 = ne2.copy()
ne3[('observation', 'id', '')] = 5
ne3 = ne3.set_index([('set', 'date', ''), ('point', 'id', ''), ('observation', 'id', '')])
ne3.head()

scope                                                     index              \
variable                                                                      
('natural_enemy_count', 'name')                              e2  e3  e4  e5   
(set, date, )           (point, id, ) (observation, id, )                     
2017-07-14 12:31:24.194 0.0           5                       6   7   8   9   
                        1.0           5                      20  21  22  23   
                        2.0           5                      34  35  36  37   
                        3.0           5                      48  49  50  51   
                        4.0           5                      62  63  64  65   

scope                                                                      \
variable                                                                    
('natural_enemy_count', 'name')                            e6  e7  e8  e9   
(set, date, )           (point, id, ) (observation, id, )                   
2017-07-14 12:31:24.194 0.0           5                    10  11  12  13   
                        1.0           5                    24  25  26  27   
                        2.0           5                    38  39  40  41   
                        3.0           5                    52  53  54  55   
                        4.0           5                    66  67  68  69   

scope                                                     observation       \
variable                                                     complete        
('natural_enemy_count', 'name')                                    e2   e3   
(set, date, )           (point, id, ) (observation, id, )                    
2017-07-14 12:31:24.194 0.0           5                           0.0  0.0   
                        1.0           5                           0.0  0.0   
                        2.0           5                           0.0  0.0   
                        3.0           5                           0.0  0.0   
                        4.0           5                           0.0  0.0   

scope                                                       ...     \
variable                                                    ...      
('natural_enemy_count', 'name')                             ...      
(set, date, )           (point, id, ) (observation, id, )   ...      
2017-07-14 12:31:24.194 0.0           5                     ...      
                        1.0           5                     ...      
                        2.0           5                     ...      
                        3.0           5                     ...      
                        4.0           5                     ...      

scope                                                     natural_enemy_count  \
variable                                                                count   
('natural_enemy_count', 'name')                                            e3   
(set, date, )           (point, id, ) (observation, id, )                       
2017-07-14 12:31:24.194 0.0           5                                   0.0   
                        1.0           5                                   0.0   
                        2.0           5                                   0.0   
                        3.0           5                                   0.0   
                        4.0           5                                   0.0   

scope                                                                          \
variable                                                                        
('natural_enemy_count', 'name')                             e4   e5   e6   e7   
(set, date, )           (point, id, ) (observation, id, )                       
2017-07-14 12:31:24.194 0.0           5                    0.0  0.0  0.0  0.0   
                        1.0           5                    0.0  0.0  0.0  0.0   
                        2.0           5     

In [24]:
ne3 = ne3[['natural_enemy_count']]
ne3.head()

scope                                                     natural_enemy_count  \
variable                                                                count   
('natural_enemy_count', 'name')                                            e1   
(set, date, )           (point, id, ) (observation, id, )                       
2017-07-14 12:31:24.194 0.0           5                                   1.0   
                        1.0           5                                   0.0   
                        2.0           5                                   0.0   
                        3.0           5                                   0.0   
                        4.0           5                                   0.0   

scope                                                                          \
variable                                                                        
('natural_enemy_count', 'name')                             e2   e3   e4   e5   
(set, date, )           (point, id, ) (observation, id, )                       
2017-07-14 12:31:24.194 0.0           5                    1.0  0.0  0.0  0.0   
                        1.0           5                    0.0  0.0  0.0  0.0   
                        2.0           5                    0.0  0.0  0.0  0.0   
                        3.0           5                    0.0  0.0  0.0  0.0   
                        4.0           5                    0.0  0.0  0.0  0.0   

scope                                                                          
variable                                                                       
('natural_enemy_count', 'name')                             e6   e7   e8   e9  
(set, date, )           (point, id, ) (observation, id, )                      
2017-07-14 12:31:24.194 0.0           5                    0.0  0.0  0.0  0.0  
                        1.0           5                    0.0  0.0  0.0  0.0  
                        2.0           5                    0.0  0.0  0.0  0.0  
                        3.0           5                    0.0  0.0  0.0  0.0  
                        4.0           5                    0.0  0.0  0.0  0.0

In [25]:
ne3.index = ne3.index.set_names([ ('set', 'date'), ('point', 'id'), ('observation', 'id')])
ne3.head()

scope                                                 natural_enemy_count  \
variable                                                            count   
('natural_enemy_count', 'name')                                        e1   
(set, date)             (point, id) (observation, id)                       
2017-07-14 12:31:24.194 0.0         5                                 1.0   
                        1.0         5                                 0.0   
                        2.0         5                                 0.0   
                        3.0         5                                 0.0   
                        4.0         5                                 0.0   

scope                                                                      \
variable                                                                    
('natural_enemy_count', 'name')                         e2   e3   e4   e5   
(set, date)             (point, id) (observation, id)                       
2017-07-14 12:31:24.194 0.0         5                  1.0  0.0  0.0  0.0   
                        1.0         5                  0.0  0.0  0.0  0.0   
                        2.0         5                  0.0  0.0  0.0  0.0   
                        3.0         5                  0.0  0.0  0.0  0.0   
                        4.0         5                  0.0  0.0  0.0  0.0   

scope                                                                      
variable                                                                   
('natural_enemy_count', 'name')                         e6   e7   e8   e9  
(set, date)             (point, id) (observation, id)                      
2017-07-14 12:31:24.194 0.0         5                  0.0  0.0  0.0  0.0  
                        1.0         5                  0.0  0.0  0.0  0.0  
                        2.0         5                  0.0  0.0  0.0  0.0  
                        3.0         5                  0.0  0.0  0.0  0.0  
                        4.0         5                  0.0  0.0  0.0  0.0

#### flatten unstacked column names

In [17]:
new_names2 = [tuple([c[0], c[2] or c[1], None]) for c in ne3.columns.values]
ne3.columns = pandas.MultiIndex.from_tuples(new_names2).droplevel(2)
ne3

natural_enemy_count  \
                                                                       e1   
(set, date)             (point, id) (observation, id)                       
2017-07-14 12:31:24.194 0.0         5                                 1.0   
                        1.0         5                                 0.0   
                        2.0         5                                 0.0   
                        3.0         5                                 0.0   
                        4.0         5                                 0.0   
2017-07-18 10:31:22.263 0.0         5                                 0.0   
                        1.0         5                                 0.0   
                        2.0         5                                 0.0   
                        3.0         5                                 0.0   
                        4.0         5                                 0.0   
2017-07-18 10:51:30.195 0.0         5                                 0.0   
                        1.0         5                                 0.0   
                        2.0         5                                 0.0   
                        3.0         5                                 0.0   
                        4.0         5                                 0.0   
2017-07-18 11:39:46.221 0.0         5                                 0.0   
                        1.0         5                                 0.0   
                        2.0         5                                 0.0   
                        3.0         5                                 0.0   
                        4.0         5                                 0.0   
2017-07-18 14:25:46.769 0.0         5                                 0.0   
                        1.0         5                                 0.0   
                        2.0         5                                 0.0   
                        3.0         5                                 0.0   
                        4.0         5                                 0.0   
2017-07-18 14:49:08.831 0.0         5                                 0.0   
                        1.0         5                                 0.0   
                        2.0         5                                 0.0   
                        3.0         5                                 0.0   
                        4.0         5                                 0.0   
...                                                                   ...   
2017-08-24 13:10:20.476 5.0         5                                 0.0   
                        6.0         5                                 0.0   
                        7.0         5                                 0.0   
                        8.0         5                                 0.0   
                        9.0         5                                 0.0   
2017-08-24 16:57:43.540 0.0         5                                 1.0   
                        1.0         5                                 0.0   
                        2.0         5                                 0.0   
                        3.0         5                                 0.0   
                        4.0         5                                 0.0   
2017-08-25 12:54:07.934 0.0         5                                 4.0   
                        1.0         5                                 0.0   
                        2.0         5                                 2.0   
                        3.0         5                                 4.0   
                        4.0         5                                 2.0   
                        5.0         5                                 2.0   
                        6.0         5                                 0.0   
                        7.0         5                                 3.0   
                        8.0         5             

### remainder

In [18]:
remainder_column_names = list(set(df.columns) - set(base_df.columns) - set(aphid_columns) - set(ne_columns) | set(index_columns))
remainder_df = df.loc[
    df[('observation', 'aphid_subtotal')].notna() | df[('point', 'natural_enemy_subtotal')].notna(),
    remainder_column_names
].sort_index(axis='columns').set_index(index_columns)
remainder_df


scope                                                    observation  \
variable                                              aphid_subtotal   
(set, date)             (point, id) (observation, id)                  
2017-07-14 12:31:24.194 0.0         0.0                          0.0   
                                    1.0                          0.0   
                                    2.0                          0.0   
                                    3.0                          0.0   
                                    4.0                          0.0   
                                    5.0                          NaN   
                        1.0         0.0                          0.0   
                                    1.0                          0.0   
                                    2.0                          0.0   
                                    3.0                          0.0   
                                    4.0                          0.0   
                                    5.0                          NaN   
                        2.0         0.0                          0.0   
                                    1.0                          0.0   
                                    2.0                          0.0   
                                    3.0                          0.0   
                                    4.0                          0.0   
                                    5.0                          NaN   
                        3.0         0.0                          0.0   
                                    1.0                          0.0   
                                    2.0                          0.0   
                                    3.0                          0.0   
                                    4.0                          0.0   
                                    5.0                          NaN   
                        4.0         0.0                          0.0   
                                    1.0                          0.0   
                                    2.0                          0.0   
                                    3.0                          0.0   
                                    4.0                          0.0   
                                    5.0                          NaN   
...                                                              ...   
2017-08-25 13:05:14.062 5.0         0.0                          0.0   
                                    1.0                          0.0   
                                    2.0                          0.0   
                                    3.0                          0.0   
                                    4.0                          0.0   
                                    5.0                          NaN   
                        6.0         0.0                          0.0   
                                    1.0                          0.0   
                                    2.0                          0.0   
                                    3.0                          0.0   
                                    4.0                          0.0   
                                    5.0                          NaN   
                        7.0         0.0                          0.0   
                                    1.0                          0.0   
                                    2.0                          0.0   
                                    3.0                          0.0   
                                    4.0                          0.0   
                                    5.0                          NaN   
                        8.0         0.0                          0.0   
                                    1.0                          0.0   
                                    2.0                          0.0   
                                    3.0 

## merge parts

In [19]:
reindexed_base = base_df.reset_index(
    level=[
        ('observation', 'id'),
        ('point', 'id')
    ],
    drop=True
).drop_duplicates()

In [20]:
joined_sum = index_df.join(
    aphid_df
).join(
    ne3[['natural_enemy_count']]
).join(
    remainder_df
).sum(
    level=[('set', 'date'), ('point', 'id')]
).join(
    reindexed_base
).reset_index()
joined_sum

scope                        set point observation          aphid_count       \
variable                    date    id    complete disabled          a1   a2   
0        2017-07-14 12:31:24.194   0.0         0.0      0.0         7.0  0.0   
1        2017-07-14 12:31:24.194   1.0         0.0      0.0         0.0  0.0   
2        2017-07-14 12:31:24.194   2.0         0.0      0.0         0.0  0.0   
3        2017-07-14 12:31:24.194   3.0         0.0      0.0         0.0  0.0   
4        2017-07-14 12:31:24.194   4.0         0.0      0.0         0.0  0.0   
5        2017-07-18 10:31:22.263   0.0         5.0      0.0         0.0  0.0   
6        2017-07-18 10:31:22.263   1.0         5.0      0.0         0.0  0.0   
7        2017-07-18 10:31:22.263   2.0         5.0      0.0         6.0  0.0   
8        2017-07-18 10:31:22.263   3.0         5.0      0.0         0.0  0.0   
9        2017-07-18 10:31:22.263   4.0         5.0      0.0         0.0  0.0   
10       2017-07-18 10:51:30.195   0.0         5.0      0.0         2.0  0.0   
11       2017-07-18 10:51:30.195   1.0         5.0      0.0         0.0  0.0   
12       2017-07-18 10:51:30.195   2.0         5.0      0.0         0.0  0.0   
13       2017-07-18 10:51:30.195   3.0         5.0      0.0         0.0  0.0   
14       2017-07-18 10:51:30.195   4.0         5.0      0.0         0.0  0.0   
15       2017-07-18 11:39:46.221   0.0         5.0      0.0         0.0  0.0   
16       2017-07-18 11:39:46.221   1.0         5.0      0.0         0.0  0.0   
17       2017-07-18 11:39:46.221   2.0         5.0      0.0         0.0  0.0   
18       2017-07-18 11:39:46.221   3.0         5.0      0.0         0.0  0.0   
19       2017-07-18 11:39:46.221   4.0         5.0      0.0         0.0  0.0   
20       2017-07-18 14:25:46.769   0.0         5.0      0.0         0.0  0.0   
21       2017-07-18 14:25:46.769   1.0         5.0      0.0         0.0  0.0   
22       2017-07-18 14:25:46.769   2.0         5.0      0.0         0.0  0.0   
23       2017-07-18 14:25:46.769   3.0         5.0      0.0         0.0  0.0   
24       2017-07-18 14:25:46.769   4.0         5.0      0.0         0.0  0.0   
25       2017-07-18 14:49:08.831   0.0         5.0      0.0         0.0  0.0   
26       2017-07-18 14:49:08.831   1.0         5.0      0.0         0.0  0.0   
27       2017-07-18 14:49:08.831   2.0         5.0      0.0         0.0  0.0   
28       2017-07-18 14:49:08.831   3.0         5.0      0.0         0.0  0.0   
29       2017-07-18 14:49:08.831   4.0         5.0      0.0         0.0  0.0   
..                           ...   ...         ...      ...         ...  ...   
305      2017-08-24 13:10:20.476   5.0         5.0      0.0         2.0  0.0   
306      2017-08-24 13:10:20.476   6.0         5.0      0.0         0.0  1.0   
307      2017-08-24 13:10:20.476   7.0         5.0      0.0         0.0  0.0   
308      2017-08-24 13:10:20.476   8.0         5.0      0.0         0.0  0.0   
309      2017-08-24 13:10:20.476   9.0         5.0      0.0         0.0  0.0   
310      2017-08-24 16:57:43.540   0.0         5.0      0.0        15.0  0.0   
311      2017-08-24 16:57:43.540   1.0         5.0      0.0        53.0  0.0   
312      2017-08-24 16:57:43.540   2.0         5.0      0.0         7.0  0.0   
313      2017-08-24 16:57:43.540   3.0         5.0      0.0         5.0  9.0   
314      2017-08-24 16:57:43.540   4.0         5.0      0.0         3.0  0.0   
315      2017-08-25 12:54:07.934   0.0         5.0      0.0         0.0  2.0   
316      2017-08-25 12:54:07.934   1.0         5.0      0.0         2.0  1.0   
317      2017-08-25 12:54:07.934   2.0         5.0      0.0         4.0  3.0   
318      2017-08-25 12:54:07.934   3.0         5.0      0.0         3.0  2.0   
319      2017-08-25 12:54:07.934   4.0         5.0      0.0         0.0  0.0   
320      2017-08-25 12:54:07.934   5.0         5.0      0.0         0.0  0.0   
321      2017-08-25 12:54:07.934   6.0         5.0      0.0         0.0  0.0   


In [21]:
joined_df = index_df.join(
    aphid_df
).join(
    ne3[['natural_enemy_count']]
).join(
    remainder_df
).join(
    base_df
).reset_index()
joined_df

scope                        set point observation                    \
variable                    date    id          id complete disabled   
0        2017-07-14 12:31:24.194   0.0         0.0      0.0      0.0   
1        2017-07-14 12:31:24.194   0.0         1.0      0.0      0.0   
2        2017-07-14 12:31:24.194   0.0         2.0      0.0      0.0   
3        2017-07-14 12:31:24.194   0.0         3.0      0.0      0.0   
4        2017-07-14 12:31:24.194   0.0         4.0      0.0      0.0   
5        2017-07-14 12:31:24.194   0.0         5.0      NaN      NaN   
6        2017-07-14 12:31:24.194   1.0         0.0      0.0      0.0   
7        2017-07-14 12:31:24.194   1.0         1.0      0.0      0.0   
8        2017-07-14 12:31:24.194   1.0         2.0      0.0      0.0   
9        2017-07-14 12:31:24.194   1.0         3.0      0.0      0.0   
10       2017-07-14 12:31:24.194   1.0         4.0      0.0      0.0   
11       2017-07-14 12:31:24.194   1.0         5.0      NaN      NaN   
12       2017-07-14 12:31:24.194   2.0         0.0      0.0      0.0   
13       2017-07-14 12:31:24.194   2.0         1.0      0.0      0.0   
14       2017-07-14 12:31:24.194   2.0         2.0      0.0      0.0   
15       2017-07-14 12:31:24.194   2.0         3.0      0.0      0.0   
16       2017-07-14 12:31:24.194   2.0         4.0      0.0      0.0   
17       2017-07-14 12:31:24.194   2.0         5.0      NaN      NaN   
18       2017-07-14 12:31:24.194   3.0         0.0      0.0      0.0   
19       2017-07-14 12:31:24.194   3.0         1.0      0.0      0.0   
20       2017-07-14 12:31:24.194   3.0         2.0      0.0      0.0   
21       2017-07-14 12:31:24.194   3.0         3.0      0.0      0.0   
22       2017-07-14 12:31:24.194   3.0         4.0      0.0      0.0   
23       2017-07-14 12:31:24.194   3.0         5.0      NaN      NaN   
24       2017-07-14 12:31:24.194   4.0         0.0      0.0      0.0   
25       2017-07-14 12:31:24.194   4.0         1.0      0.0      0.0   
26       2017-07-14 12:31:24.194   4.0         2.0      0.0      0.0   
27       2017-07-14 12:31:24.194   4.0         3.0      0.0      0.0   
28       2017-07-14 12:31:24.194   4.0         4.0      0.0      0.0   
29       2017-07-14 12:31:24.194   4.0         5.0      NaN      NaN   
...                          ...   ...         ...      ...      ...   
1980     2017-08-25 13:05:14.062   5.0         0.0      1.0      0.0   
1981     2017-08-25 13:05:14.062   5.0         1.0      1.0      0.0   
1982     2017-08-25 13:05:14.062   5.0         2.0      1.0      0.0   
1983     2017-08-25 13:05:14.062   5.0         3.0      1.0      0.0   
1984     2017-08-25 13:05:14.062   5.0         4.0      1.0      0.0   
1985     2017-08-25 13:05:14.062   5.0         5.0      NaN      NaN   
1986     2017-08-25 13:05:14.062   6.0         0.0      1.0      0.0   
1987     2017-08-25 13:05:14.062   6.0         1.0      1.0      0.0   
1988     2017-08-25 13:05:14.062   6.0         2.0      1.0      0.0   
1989     2017-08-25 13:05:14.062   6.0         3.0      1.0      0.0   
1990     2017-08-25 13:05:14.062   6.0         4.0      1.0      0.0   
1991     2017-08-25 13:05:14.062   6.0         5.0      NaN      NaN   
1992     2017-08-25 13:05:14.062   7.0         0.0      1.0      0.0   
1993     2017-08-25 13:05:14.062   7.0         1.0      1.0      0.0   
1994     2017-08-25 13:05:14.062   7.0         2.0      1.0      0.0   
1995     2017-08-25 13:05:14.062   7.0         3.0      1.0      0.0   
1996     2017-08-25 13:05:14.062   7.0         4.0      1.0      0.0   
1997     2017-08-25 13:05:14.062   7.0         5.0      NaN      NaN   
1998     2017-08-25 13:05:14.062   8.0         0.0      1.0      0.0   
1999     2017-08-25 13:05:14.062   8.0         1.0      1.0      0.0   
2000     2017-08-25 13:05:14.062   8.0         2.0      1.0      0.0   
2001     2017-08-25 13:05:14.062   8.0         3.0      1.0      0.0   
2002     2017-08-25 13:05:14.062   8.0  

# save file

In [22]:
joined_sum.columns = joined_sum.columns.map('_'.join)
joined_df.columns = joined_df.columns.map('_'.join)
with pandas.ExcelWriter('data/real/2017/CAM data from iPads/out_df.xlsx') as file_writer:
    joined_sum.to_excel(
        file_writer,
        sheet_name='2017 - CAM - sum',
        freeze_panes=(1, 1),
        index_label='row'
    )
    joined_df.to_excel(
        file_writer,
        sheet_name='2017 - CAM - detailed',
        freeze_panes=(1, 1),
        index_label='row'
    )
    file_writer.save()